In [1]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [2]:
import os
import re
import ast
import pandas as pd

def parse_metrics(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    folder_name = os.path.basename(os.path.dirname(file_path))
    metrics = {
        "Folder": folder_name,
        "Path": file_path,
    }

    regex_patterns = {
        "train_loss": r"train_loss:\s*([^\n]+)",
        "train_mse": r"train_mse:\s*([^\n]+)",
        "train_mae": r"train_mae:\s*([^\n]+)",
        "train_rmse": r"train_rmse:\s*([^\n]+)",
        "train_r2": r"train_r2:\s*([^\n]+)",
        "val_loss": r"val_loss:\s*([^\n]+)",
        "val_mse": r"val_mse:\s*([^\n]+)",
        "val_mae": r"val_mae:\s*([^\n]+)",
        "val_rmse": r"val_rmse:\s*([^\n]+)",
        "val_r2": r"val_r2:\s*([^\n]+)",
        "test_loss": r"test_loss:\s*([^\n]+)",
        "test_mse": r"test_mse:\s*([^\n]+)",
        "test_mae": r"test_mae:\s*([^\n]+)",
        "test_rmse": r"test_rmse:\s*([^\n]+)",
        "test_r2": r"test_r2:\s*([^\n]+)",
        "min_lr": r"min_lr:\s*([^\n]+)",
        "max_lr": r"max_lr:\s*([^\n]+)",
        "total_time": r"total_time:\s*([^\n]+)",
        "average_epoch_time": r"average_epoch_time:\s*([^\n]+)",
    }

    for key, pattern in regex_patterns.items():
        match = re.search(pattern, content)
        if match:
            value = match.group(1).strip()
            try:
                value = ast.literal_eval(value)
            except:
                pass
            metrics[key] = value
        else:
            metrics[key] = None

    return metrics

def find_and_parse_metrics(root_folder):
    metrics_list = []
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower() == "metrics.txt":
                full_path = os.path.join(dirpath, filename)
                metrics = parse_metrics(full_path)
                metrics_list.append(metrics)
    return metrics_list

def save_to_excel(metrics_list, output_path="metrics_summary.xlsx"):
    df = pd.DataFrame(metrics_list)
    df.to_excel(output_path, index=False)
    print(f"Saved {len(df)} entries to {output_path}")


In [3]:
root_folder = "./"  # Change this to your root directory if needed
metrics_list = find_and_parse_metrics(root_folder)
save_to_excel(metrics_list)

Saved 102 entries to metrics_summary.xlsx
